In [1]:
import numpy as np
import pandas as pd
import tarfile
from bokeh.io import curdoc, vform, output_notebook, push_notebook
from bokeh.models import ColumnDataSource, HBox, VBox
from bokeh.models.widgets import Slider, Button
from bokeh.plotting import Figure,show
from ipywidgets import interact


In [2]:
def get_reconstruction(eigenvectors, m_projection, Mean):
    k = len(m_projection)
    U = eigenvectors[:k]
    Eig = np.matrix(U.T)
    rec = np.array(m_projection * Eig.transpose() + Mean)
    rec = np.ravel(rec)
    return rec


eigfilepath= './data/weekday/total_flow_weekday_eigenvectors.pivot_2009_grouping_pca_tmp.csv'
meanvecpath= './data/weekday/total_flow_weekday_mean_vector.pivot_2009_grouping_pca_tmp.csv'
sourcetranspath= './data/weekday/total_flow_transformed.pivot_2009_grouping_weekday_pca_transform_tmp.csv.tar.gz'
transpath= '../total_flow_transformed.pivot_2009_grouping_weekday_pca_transform_tmp.csv'


with tarfile.open(sourcetranspath) as tar:
    tar.extractall()

m_eigs = pd.read_csv(eigfilepath, header=None).values
m_eigs_t = m_eigs.T
m_mean_vector = pd.read_csv(meanvecpath, header=None).values[0]
m_df = pd.read_csv(transpath, header=None)

v0pos= 6
v1pos= 7
v2pos= 8
v3pos= 9

v0_struct = (np.min(m_df[v0pos]), np.max(m_df[v0pos]))
v1_struct = (np.min(m_df[v1pos]), np.max(m_df[v1pos]))
v2_struct = (np.min(m_df[v2pos]), np.max(m_df[v2pos]))
v3_struct = (np.min(m_df[v3pos]), np.max(m_df[v3pos]))

output_notebook()

# Set up data
m_projection = [0., 0., 0., 0.]
rec = get_reconstruction(m_eigs_t, m_projection, m_mean_vector)
x = [i for i in xrange(np.shape(rec)[0])]
y = [v for v in rec]
source = ColumnDataSource(data=dict(x=x, y=y))

# Set up plot
plot = Figure(plot_height=600, plot_width=1000, title="Eigenvector Analysis",
              tools="",
              x_range=[0, 289], y_range=[0, 1500.], x_axis_label='Time', y_axis_label='Veh/5m')
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# Set up callbacks
def update_data(v0_value, v1_value, v2_value, v3_value):

    # Generate the new curve
    m_projection = [v0_value, v1_value, v2_value, v3_value]
    rec = get_reconstruction(m_eigs_t, m_projection, m_mean_vector)
    x = [i for i in xrange(np.shape(rec)[0])]
    y = [v for v in rec]

    source.data = dict(x=x, y=y)
    push_notebook()

show(plot, notebook_handle=True)

Loading BokehJS ...

In [3]:
interact(update_data, v0_value = (-15000,15000), v1_value= (-12000,12000), v2_value= (-6000,6000), 
         v3_value= (-10000,10000))